In [2]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import re
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords


from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tharakan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tharakan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tharakan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
engine = create_engine('sqlite:///elog_data_2011.db')
df = pd.read_sql_table('elog_data_2011', engine)

In [18]:
df.tail(10)

,elogid,title,text,tag,title_and_text
6903,554308,"Laser heater motor delay has lost ""home"" position",Put in Laser heater for diagnostics and switch...,LCLS,"Laser heater motor delay has lost ""home"" posit..."
6904,554298,Garth at CXI is seeing lower Intensity than GD...,Garth is hoping that Jim Turner identified the...,LCLS,Garth at CXI is seeing lower Intensity than GD...
6905,554292,Jim Craft advisory on CR2 LI21,Jim Craft has left a note on CR 2 of LI21 that...,LCLS,Jim Craft advisory on CR2 LI21 Jim Craft has l...
6906,554290,"repointUndulatorLine(0,0,-45,-45)",,LCLS,"repointUndulatorLine(0,0,-45,-45)"
6907,554265,Switching to Matlab UND launch. Fast Und launc...,,LCLS,Switching to Matlab UND launch. Fast Und launc...
6908,554263,"Model manager hangs on ""gold"" stage",,LCLS,"Model manager hangs on ""gold"" stage"
6909,554261,BBA Round 2 complete,,LCLS,BBA Round 2 complete
6910,554258,Swapped 30-6 for 30-4,Will LEM the change in; seems to be alright so...,LCLS,Swapped 30-6 for 30-4 Will LEM the change in; ...
6911,554257,1st Round BBA done,,LCLS,1st Round BBA done
6912,554250,Emittance Measurements - 9491 eV (14.673 GeV...,<table class=emittanceTable>\n<tr><th></th><th...,LCLS,Emittance Measurements - 9491 eV (14.673 GeV...


> Questions to be answere later: Does it perform better all lower or all uppercase??

In [21]:
# Turning it all to lower case and removing newline stuff

df['title_and_text'] = df['title_and_text'].str.lower()
df['title_and_text'] = df['title_and_text'].str.replace('\n', ' ')
df.tail(10)

,elogid,title,text,tag,title_and_text
6903,554308,"Laser heater motor delay has lost ""home"" position",Put in Laser heater for diagnostics and switch...,LCLS,"laser heater motor delay has lost ""home"" posit..."
6904,554298,Garth at CXI is seeing lower Intensity than GD...,Garth is hoping that Jim Turner identified the...,LCLS,garth at cxi is seeing lower intensity than gd...
6905,554292,Jim Craft advisory on CR2 LI21,Jim Craft has left a note on CR 2 of LI21 that...,LCLS,jim craft advisory on cr2 li21 jim craft has l...
6906,554290,"repointUndulatorLine(0,0,-45,-45)",,LCLS,"repointundulatorline(0,0,-45,-45)"
6907,554265,Switching to Matlab UND launch. Fast Und launc...,,LCLS,switching to matlab und launch. fast und launc...
6908,554263,"Model manager hangs on ""gold"" stage",,LCLS,"model manager hangs on ""gold"" stage"
6909,554261,BBA Round 2 complete,,LCLS,bba round 2 complete
6910,554258,Swapped 30-6 for 30-4,Will LEM the change in; seems to be alright so...,LCLS,swapped 30-6 for 30-4 will lem the change in; ...
6911,554257,1st Round BBA done,,LCLS,1st round bba done
6912,554250,Emittance Measurements - 9491 eV (14.673 GeV...,<table class=emittanceTable>\n<tr><th></th><th...,LCLS,emittance measurements - 9491 ev (14.673 gev...


In [23]:
df.tag.value_counts()

LCLS     5631
FACET    1282
Name: tag, dtype: int64

In [24]:
X = df['title_and_text'] 
Y = df.drop(['text','title','elogid', 'title_and_text'], axis = 1)

In [31]:
# Creat tokenizer function
def tokenize(x):
    
    # Generating list of stop words
    stop_words = set(stopwords.words('english'))
    
    # Separate sentance into individual words
    no_punctuation_x = re.sub(r"[^a-zA-Z0-9]"," ", x)
    word_token = word_tokenize(no_punctuation_x)
    
    # Lemmatizing each word and added cleaned words to clean_words
    lemmatizer = WordNetLemmatizer()
    clean_words = []
    for word in word_token:
        clean_words.append(lemmatizer.lemmatize(word.lower().strip()))

    # Return lematized words that are indeed words and are not in stopwords list
    final_token = [w for w in clean_words if w not in stop_words]
    return final_token

In [33]:
# Proof that it works. Watch out for cells with tables tho, there is a lot of useless information in them.
#tokenize(df.at[6912,'title_and_text'])

In [34]:
pipeline = Pipeline([
    ('vectorizer', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))    
])

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X,Y)
pipeline.fit(X_train, y_train)